# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = "../cities_output2.csv"

city_data_df = pd.read_csv(r'../cities_output2.csv')
city_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed
0,victoria,100,HK,1635208359,75,22.2855,114.1577,77.25,1.01
1,albany,100,US,1635208533,90,42.6001,-73.9662,56.52,4.45
2,kruisfontein,39,ZA,1635208638,95,-34.0033,24.7314,60.89,5.08
3,guerrero negro,0,MX,1635208638,75,27.9769,-114.0611,66.99,17.34
4,rikitea,100,PF,1635208639,71,-23.1203,-134.9692,75.63,18.14


In [3]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562 entries, 0 to 561
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   City          562 non-null    object 
 1   Cloudiness    562 non-null    int64  
 2   Country       558 non-null    object 
 3   Date          562 non-null    int64  
 4   Humidity      562 non-null    int64  
 5   Lat           562 non-null    float64
 6   Lng           562 non-null    float64
 7   Max Temp (F)  562 non-null    float64
 8   Wind Speed    562 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 39.6+ KB


In [4]:
city_data_df['Country'][0]

'HK'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=figure_layout)

marker_locations = city_data_df[['Lat', 'Lng']]

markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

In [6]:
humidity_list = city_data_df["Humidity"].tolist()


In [7]:
fig = gmaps.figure(layout =figure_layout)

heat_layer = gmaps.heatmap_layer(marker_locations, weights=humidity_list, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
perfect_weather=city_data_df.loc[(city_data_df["Max Temp (F)"]>=75)&(city_data_df["Max Temp (F)"]<=80)&(city_data_df["Humidity"]<=35)&(city_data_df["Wind Speed"]<=10)]

perfect_weather


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp (F),Wind Speed
14,rio cuarto,0,AR,1635208520,34,-33.1307,-64.3499,78.57,9.22
38,airai,67,TL,1635208650,34,-8.9266,125.4092,75.97,2.62
199,salta,0,AR,1635208713,28,-24.7859,-65.4117,77.29,4.61
260,saint george,1,US,1635208738,29,37.1041,-113.5841,77.47,3.00
504,inverell,0,AU,1635208836,32,-29.7833,151.1167,78.12,4.00
525,moree,0,AU,1635208844,16,-29.4667,149.8500,77.02,5.75


In [9]:
hotel_df = perfect_weather

In [10]:
hotelname=[]
index=0

for index, row in hotel_df.iterrows():
    params = {
        "radius": 30000,
        "rankby": "prominence",
        "keyword": "hotel",
        "key": g_key}
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_json = requests.get(url, params=params).json()
    hotelname.append(hotel_json['results'][0]['name'])
hotel_df["Hotel Name"]=hotelname


<ipython-input-10-5006fd2e6165>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=hotelname


In [11]:
len(hotelname)

6

In [12]:
pprint(hotel_json)

{'html_attributions': [],
 'next_page_token': 'Aap_uEAgYdy1btTqmCVViFfW-hMx17bYsi4TtqCclnW_rTe6OmrUfTFybiKIWiWMnCxnNbwLDOuGx4UDvbnCu_0tddMTHpVk97y6KrXc4SdOoEf7YlD3bVbEubOCPICYiibDwWbrwJ_sYrRTpWOdaKmQySxzg9gdiBTK0Ow4va4IixL-3L9TwPEQUdn2FzFnG1AihPuxzwEbwsGFh9Ce3tuh9TclfrhDeujV31yVDBKN4hO4d2j-g6cjupswVD6YRiINs4gKHWmI9COL62efiyG6Kmu0QYZJQEA67ojifo5-Xg3s7KKqu_QbgR-IJu3DoTa7BoeVgzWnsHIsdBNjlcHrl-LLthG_j99roGn0WuhvMEtttbhoMhoeC04_YMhg3A_v1iuISy4xkdO0TpcvP_YIGXd_kSO1jU4I7ZCpY-af2KUO_r7OfFOKVfev',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -29.4647409, 'lng': 149.8482918},
                           'viewport': {'northeast': {'lat': -29.46339117010728,
                                                      'lng': 149.8498219798927},
                                        'southwest': {'lat': -29.46609082989272,
                                                      'lng': 149.8471223201073}}},
              'icon': 'https://maps.gstatic.com/map

In [13]:
print(hotelname)

['Grand Hotel Río Cuarto', 'Hotel Umaliurai', 'Hotel Salta', 'Best Western Plus Zion West Hotel', 'Delungra Hotel', 'Albert Motel']


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Grand Hotel Río Cuarto</dd>\n<dt>City</dt><dd>rio cuarto</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Umaliurai</dd>\n<dt>City</dt><dd>airai</dd>\n<dt>Country</dt><dd>TL</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Salta</dd>\n<dt>City</dt><dd>salta</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Best Western Plus Zion West Hotel</dd>\n<dt>City</dt><dd>saint george</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Delungra Hotel</dd>\n<dt>City</dt><dd>inverell</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Albert Motel</dd>\n<dt>City</dt><dd>moree</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n']

In [16]:
# Add marker layer ontop of heat map
info_box_content = hotel_info
markers = gmaps.marker_layer(locations, hotel_info)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px', margin='0 auto 0 auto', width='500px'))